## The goal of this model is to predict airplane arrival delay 1 week in advance

In [1]:
import pandas as pd
import datetime
import psycopg2
import numpy as np

In [42]:
#Connect to database with psycopg2
connection = psycopg2.connect(
    host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com",
    database="mid_term_project",
    user="lhl_student",
    password="lhl_student",
    port=5432
)

In [43]:
#target variable is arr_delay. positive value is delay, negative value is early 
df_flights = pd.read_sql_query('SELECT fl_date, mkt_carrier, origin, dest, crs_dep_time, crs_arr_time, crs_elapsed_time, arr_delay, distance, dest_city_name FROM flights ORDER BY RANDOM() LIMIT 60000;', connection)
df_flights.head()

,fl_date,mkt_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,arr_delay,distance,dest_city_name
0,2019-07-19,AA,PHL,RDU,1230,1350,80.0,-20.0,337.0,"Raleigh/Durham, NC"
1,2018-07-11,DL,FWA,ATL,810,1003,113.0,-15.0,508.0,"Atlanta, GA"
2,2018-08-08,AA,ILM,LGA,628,819,111.0,1.0,500.0,"New York, NY"
3,2019-07-20,DL,ILM,LGA,1205,1347,102.0,-1.0,500.0,"New York, NY"
4,2018-05-22,WN,MSP,MCI,945,1110,85.0,-21.0,393.0,"Kansas City, MO"


## Data Cleaning

In [27]:
#the only nulls are in arr_delay, which we assume is on time flight
df_flights.isnull().sum()

fl_date                0
mkt_carrier            0
origin                 0
dest                   0
crs_dep_time           0
crs_arr_time           0
crs_elapsed_time       0
arr_delay           1214
distance               0
dest_city_name         0
dtype: int64

In [28]:
#only concern is elapsed_time has negative values. everything else seems realistic
df_flights.describe()

,crs_dep_time,crs_arr_time,crs_elapsed_time,arr_delay,distance
count,60000.00000,60000.000000,60000.000000,58786.000000,60000.000000
mean,1328.57880,1482.528150,137.837550,5.915881,767.419867
std,489.84092,516.375331,71.352489,53.443091,583.745954
min,4.00000,1.000000,-112.000000,-85.000000,31.000000
25%,915.00000,1100.000000,87.000000,-15.000000,342.000000
50%,1320.00000,1510.000000,119.000000,-6.000000,604.000000
75%,1735.00000,1915.000000,168.000000,8.000000,999.000000
max,2359.00000,2400.000000,705.000000,1472.000000,5095.000000


In [40]:
#how many negative elapsed time
df_flights[df_flights['crs_elapsed_time']<0]

,fl_date,mkt_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,arr_delay,distance,dest_city_name


In [30]:
#since it is just one outlier we can drop it
df_flights = df_flights[df_flights['crs_elapsed_time']>0]

## Feature Engineering

In [37]:
#getter functions
def get_day_of_week(x):
    a = int(x.split('-')[0])
    b = int(x.split('-')[1])
    c = int(x.split('-')[2])
    dt = datetime.datetime(a,b,c)
    return dt.weekday()

def get_flight_length(x):
    if x < 180:
        return 'short'
    elif x > 360:
        return 'long'
    else:
        return 'medium'
    
def get_traffic(x):
    if x < 700:
        return 'quiet'
    elif x > 4200:
        return 'busy'
    else:
        return 'moderate'
    
def get_arr_delay(x):
    if x > 0:
        return 'delay'
    elif x == 0:
        return 'on time'
    else:
        return 'early'
    
def get_season(x):
    if x > 5 and x < 9:
        return 'summer'
    elif x > 8 and x < 12:
        return 'fall'
    elif x < 6 and x > 2:
        return 'spring'
    else:
        return 'winter'

def get_dep_time_cat(x):
    if x > 1 and x < 10:
        return 'morning'
    elif x > 9 and x < 18:
        return 'afternoon'
    else:
        return 'evening'

In [35]:
df_traffic = pd.read_sql_query('SELECT origin, dest, passengers, month FROM passengers ORDER BY RANDOM() LIMIT 60000;', connection)
df_traffic.head()

,origin,dest,passengers,month
0,BNA,SFO,936.0,10
1,SJC,DEN,1758.0,2
2,MCO,LGW,2975.0,12
3,ORD,LHR,11081.0,11
4,STL,DEN,4332.0,10


In [38]:
#create features based on bining into categorical variables
df_features = df_flights.copy()[['mkt_carrier','origin','dest','arr_delay','distance']]
df_features['month'] = df_flights['fl_date'].apply(lambda x: int(x.split('-')[1]))
df_features['day_of_week'] = df_flights['fl_date'].apply(get_day_of_week)
df_features['departure_time'] = df_flights['crs_dep_time'].apply(lambda x: x//100)
df_features['flight_length'] = df_flights['crs_elapsed_time'].apply(get_flight_length)
df_features = pd.merge(df_features,df_traffic, on=['origin','dest','month'])
df_features['passengers'] = df_features['passengers'].replace(0, np.nan).dropna(axis=0, how='any')
df_features['airport_traffic'] = df_features['passengers'].apply(get_traffic)
df_features = df_features.drop(columns=['passengers'])
df_features['distance'] = df_features['distance'].apply(lambda x: x//10)
df_features['arr_delay'] = df_features['arr_delay'].apply(get_arr_delay)
df_features['weekday_weekend'] = df_features['day_of_week'].apply(lambda x: 'weekend' if x > 4 else 'weekday')
df_features['season'] = df_features['month'].apply(get_season)
df_features['departure_time_cat'] = df_features['departure_time'].apply(get_dep_time_cat)
df_features.head()

,mkt_carrier,origin,dest,arr_delay,distance,month,day_of_week,departure_time,flight_length,airport_traffic,weekday_weekend,season,departure_time_cat
0,AA,ORD,MCO,early,100.0,12,0,11,short,quiet,weekday,winter,afternoon
1,NK,ORD,MCO,early,100.0,12,5,11,short,quiet,weekend,winter,afternoon
2,AA,ORD,MCO,delay,100.0,12,0,7,short,quiet,weekday,winter,morning
3,NK,ORD,MCO,early,100.0,12,4,11,short,quiet,weekday,winter,afternoon
4,UA,ORD,MCO,early,100.0,12,2,11,short,quiet,weekday,winter,afternoon


## Exploratory Data Analysis

In [ ]:
s